In [1]:
# import libraries
import sys
sys.path.append('/scratch/ne2213/projects/tmp_packages')
sys.path.append('/scratch/ne2213/projects/tmp_packages/')
import torch
import numpy as np
import muspy
from sklearn.model_selection import train_test_split 

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# dataset is already downloaded (True if not downloaded)
emopia = muspy.EMOPIADataset("emopia/", download_and_extract=False)

***We preprocess the dataset of music tracks: It iterates through each track, extracting emotion labels and adjusting temporal resolution. Segments of fixed length are then extracted from each track, retaining only the top 4 notes in each time-step. These segments, along with their corresponding emotion labels and previous segments, are stored for further analysis or model training.***


***The resulting dataset consists of non-overlapping N-bar segments, each tailored to the desired temporal resolution and containing key musical information for the intended task***

In [3]:
# extract non-overlapping N-bar segments from all songs in dataset

tgt_res = 16  #time-steps per bar
bars = 4  # desired number of bars for input/output generation
sect = tgt_res*bars

# storing song segments/melodies
all_segments = []
prev_segments = []
labels = []

for track in emopia:
    # extract emotion label
    label = track.annotations[0].annotation['emo_class']
    label = int(label)
    one_hot = np.zeros(4)  
    one_hot[label - 1] = 1  # assign label (0,1,2,3)

    # adjust track to target temporal resolution
    track.adjust_resolution(target=tgt_res, factor=None, rounding='round')
    track = muspy.to_representation(track,'piano-roll')

    # count number of sections per track to extract
    num_segments = track.shape[0] // sect  

    # loop through all tracks
    for i in range(num_segments):
        segment = track[i * sect:(i + 1) * sect]  # Extract the segment
        transformed_data = np.zeros_like(segment)
        # only keep top 4 notes
        for j in range(segment.shape[0]):
            # Get the indices of the top 4 values in the row
            top_4_indices = np.argsort(segment[j])[-4:]
            # Set the top 4 entries to their respective values
            transformed_data[j, top_4_indices] = segment[j, top_4_indices]
        all_segments.append(transformed_data)
        labels.append(one_hot)
        if i == 0:
            prev_segments.append(track[-sect:])
        else:
            prev_segments.append(track[(i - 1) * sect:i * sect])

In [4]:
# vertically stack all sections
labels_array = np.stack(labels)
all_segments_array = np.stack(all_segments).reshape(-1, 1, sect, 128)
prev_segments_array = np.stack(prev_segments).reshape(-1, 1, sect, 128)

# normalize
all_segments_array = all_segments_array/all_segments_array.max()
prev_segments_array = prev_segments_array/prev_segments_array.max()

In [5]:
# split into train and test sets
rnd_st = 42
test_size = 0.2
X_tr, X_val, X_prev_tr, X_prev_val, y_tr, y_val = train_test_split(
    all_segments_array, prev_segments_array, labels_array, test_size=test_size, random_state=rnd_st)

In [6]:
print(X_tr.shape)
print(X_val.shape)

(16844, 1, 64, 128)
(4212, 1, 64, 128)


In [7]:
# save the data to directory
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/X_tr.npy',X_tr)
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/X_val.npy',X_val)
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/X_prev_tr.npy',X_prev_tr)
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/X_prev_val.npy',X_prev_val)
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/y_tr.npy',y_tr)
np.save('/scratch/ne2213/projects/DL/DL-FinalProject/GAN/y_val.npy',y_val)